In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib as plt
from PIL import Image
from torchvision.utils import save_image
import torchvision.datasets as datasets
import os

# Path
if not os.path.exists('./dc_img'):
    os.mkdir('./dc_img')

# To save images
def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x

# Parameters
learning_rate = 0.001
num_epochs = 10
batch_size = 32

# To transform to tensor
transforms = torchvision.transforms.ToTensor()

# Dataset for training, validation and test sets as tensors
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transforms)
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=transforms)
mnist_trainset, mnist_validationset = torch.utils.data.random_split(mnist_trainset, [50000, 10000])

# Data loader for train, test and validation sets
trainloader = torch.utils.data.DataLoader(mnist_trainset, batch_size=batch_size, num_workers=2, shuffle=True)
testloader = torch.utils.data.DataLoader(mnist_testset, batch_size=batch_size, num_workers=2, shuffle=True)
validationloader = torch.utils.data.DataLoader(mnist_validationset, batch_size=batch_size, num_workers=2, shuffle=True)

# Compute loss for test and validation
def Loss(dataLoader):
    sum_loss = 0
    for i, data in enumerate(dataloader):
        x, label = data
        inputs, labels = data
        outputs = model(inputs)
        
        # Error catching for partial batches
        dim = outputs.shape
        if dim[0] != batch_size:
            continue

        model.zero_grad()
        
        inputs = inputs.view(batch_size, -1)
        loss = loss_func(outputs, inputs)
        
        loss.backward()
        optimizer.step()
        
        sum_loss = sum_loss + loss
    
    return(sum_loss)

class autoencoder(nn.Module):

    # Defining the network
    def __init__(self):
        super(autoencoder, self).__init__()
        
        # 28*28 input channel
        self.input_size = 28*28
        self.hidden_nodes = 16
        
        # Defining the layer sizes
        self.layer1 = nn.Linear(self.input_size, self.hidden_nodes)
        self.layer2 = nn.Linear(self.hidden_nodes, self.input_size)
        
    # Forward propagation
    def forward(self, x):
        x = x.view(-1, self.input_size) # Reshaping to a vector
        
        # Encoder and Decoder
        encoded_x = self.layer1(x)
        decoded_x = self.layer2(encoded_x)
        return decoded_x

# Getting our model
model = autoencoder()

# Defining optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_func = nn.MSELoss()

# Starting Training
for epoch in range (0, num_epochs):
    epoch_loss = 0
    for i, data in enumerate(trainloader):
        inputs, labels = data
        outputs = model(inputs)
        
        # Error catching for partial batches
        dim = outputs.shape
        if dim[0] != batch_size:
            continue

        model.zero_grad()
        
        inputs = inputs.view(batch_size, -1)
        loss = loss_func(outputs, inputs)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss = epoch_loss + loss
        
    
    print("Epoch {}. Training Loss = {}".format(epoch, "%.2f" % epoch_loss))

# Saving images
for i in range(1, 4):
    input_img, _ = mnist_trainset[i]
    output_img = model(input_img)

    pic = to_img(input_img.cpu().data)
    save_image(pic, './dc_img/input_image_{}.png'.format(i))

    pic = to_img(output_img.cpu().data)
    save_image(pic, './dc_img/output_image_{}.png'.format(i))



Epoch 0. Training Loss = 65.42
Epoch 1. Training Loss = 43.94
Epoch 2. Training Loss = 42.98
Epoch 3. Training Loss = 42.90
Epoch 4. Training Loss = 42.89
Epoch 5. Training Loss = 42.89
Epoch 6. Training Loss = 42.89


NameError: name 'mnist_trainset' is not defined